### Import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

### Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

### Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

### Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)
#drop out 대신 batch norm
def batch_norm() :
    return tf.keras.layers.BatchNormalization()

### Create model

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())
        # 2->4 더 깊은 학습
        for i in range(4):
            self.model.add(dense(256, weight_init))
            self.model.add(batch_norm())
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    # weight 결정시 glorot_uniform 사용 (xavier)
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(256, weight_init))
        model.add(batch_norm())
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

### Define data & hyper-parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

### Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_batchnorm'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### Restore checkpoint & start train or test phase

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.7976, train_loss: 1.27458429, train_accuracy: 0.3281, test_Accuracy: 0.2039
Epoch: [ 0] [    1/  468] time: 2.2145, train_loss: 1.31321144, train_accuracy: 0.3516, test_Accuracy: 0.3268
Epoch: [ 0] [    2/  468] time: 2.6782, train_loss: 1.08895278, train_accuracy: 0.4453, test_Accuracy: 0.4312
Epoch: [ 0] [    3/  468] time: 3.1918, train_loss: 0.92852366, train_accuracy: 0.5000, test_Accuracy: 0.5038
Epoch: [ 0] [    4/  468] time: 3.6536, train_loss: 0.82024354, train_accuracy: 0.5000, test_Accuracy: 0.5503
Epoch: [ 0] [    5/  468] time: 4.1364, train_loss: 0.75709563, train_accuracy: 0.6328, test_Accuracy: 0.5788
Epoch: [ 0] [    6/  468] time: 4.6031, train_loss: 0.77824664, train_accuracy: 0.6016, test_Accuracy: 0.5901
Epoch: [ 0] [    7/  468] time: 5.0449, train_loss: 0.56782705, train_accuracy: 0.6016, test_Accuracy: 0.6103
Epoch: [ 0] [    8/  468] time: 5.4318,

Epoch: [ 0] [   15/  468] time: 8.7260, train_loss: 0.38196224, train_accuracy: 0.7734, test_Accuracy: 0.7853
Epoch: [ 0] [   16/  468] time: 9.2287, train_loss: 0.46127370, train_accuracy: 0.7500, test_Accuracy: 0.8058
Epoch: [ 0] [   17/  468] time: 9.6925, train_loss: 0.45606396, train_accuracy: 0.7812, test_Accuracy: 0.8248
Epoch: [ 0] [   18/  468] time: 10.1981, train_loss: 0.26240212, train_accuracy: 0.8750, test_Accuracy: 0.8386
Epoch: [ 0] [   19/  468] time: 10.6579, train_loss: 0.38536069, train_accuracy: 0.8438, test_Accuracy: 0.8501
Epoch: [ 0] [   20/  468] time: 11.1236, train_loss: 0.34420782, train_accuracy: 0.8359, test_Accuracy: 0.8575
Epoch: [ 0] [   21/  468] time: 11.5754, train_loss: 0.43986982, train_accuracy: 0.8359, test_Accuracy: 0.8639
Epoch: [ 0] [   22/  468] time: 12.0771, train_loss: 0.34234443, train_accuracy: 0.8672, test_Accuracy: 0.8708
Epoch: [ 0] [   23/  468] time: 12.5089, train_loss: 0.30537146, train_accuracy: 0.8281, test_Accuracy: 0.8729
Epoc

Epoch: [ 0] [   31/  468] time: 16.2769, train_loss: 0.34579837, train_accuracy: 0.8594, test_Accuracy: 0.8890
Epoch: [ 0] [   32/  468] time: 16.7259, train_loss: 0.22383882, train_accuracy: 0.8750, test_Accuracy: 0.8883
Epoch: [ 0] [   33/  468] time: 17.2074, train_loss: 0.34825936, train_accuracy: 0.8047, test_Accuracy: 0.8890
Epoch: [ 0] [   34/  468] time: 17.6372, train_loss: 0.22420751, train_accuracy: 0.8984, test_Accuracy: 0.8899
Epoch: [ 0] [   35/  468] time: 18.1190, train_loss: 0.29374027, train_accuracy: 0.8906, test_Accuracy: 0.8929
Epoch: [ 0] [   36/  468] time: 18.5857, train_loss: 0.33594000, train_accuracy: 0.8516, test_Accuracy: 0.8946
Epoch: [ 0] [   37/  468] time: 18.9637, train_loss: 0.19782934, train_accuracy: 0.9297, test_Accuracy: 0.8955
Epoch: [ 0] [   38/  468] time: 19.4324, train_loss: 0.29223716, train_accuracy: 0.8906, test_Accuracy: 0.8973
Epoch: [ 0] [   39/  468] time: 19.8912, train_loss: 0.28852236, train_accuracy: 0.8594, test_Accuracy: 0.9019
E

Epoch: [ 0] [   47/  468] time: 23.5255, train_loss: 0.28173313, train_accuracy: 0.9219, test_Accuracy: 0.9148
Epoch: [ 0] [   48/  468] time: 23.9842, train_loss: 0.28350955, train_accuracy: 0.8984, test_Accuracy: 0.9144
Epoch: [ 0] [   49/  468] time: 24.4072, train_loss: 0.25049716, train_accuracy: 0.8984, test_Accuracy: 0.9148
Epoch: [ 0] [   50/  468] time: 24.8720, train_loss: 0.23219606, train_accuracy: 0.9219, test_Accuracy: 0.9138
Epoch: [ 0] [   51/  468] time: 25.3785, train_loss: 0.24566206, train_accuracy: 0.8984, test_Accuracy: 0.9131
Epoch: [ 0] [   52/  468] time: 25.7166, train_loss: 0.17008822, train_accuracy: 0.9375, test_Accuracy: 0.9113
Epoch: [ 0] [   53/  468] time: 26.1724, train_loss: 0.16594297, train_accuracy: 0.9531, test_Accuracy: 0.9115
Epoch: [ 0] [   54/  468] time: 26.6850, train_loss: 0.22707237, train_accuracy: 0.9141, test_Accuracy: 0.9114
Epoch: [ 0] [   55/  468] time: 27.1847, train_loss: 0.19584805, train_accuracy: 0.9297, test_Accuracy: 0.9125
E

Epoch: [ 0] [   63/  468] time: 30.8609, train_loss: 0.18624307, train_accuracy: 0.9453, test_Accuracy: 0.9266
Epoch: [ 0] [   64/  468] time: 31.3406, train_loss: 0.22951332, train_accuracy: 0.9141, test_Accuracy: 0.9287
Epoch: [ 0] [   65/  468] time: 31.8644, train_loss: 0.30535156, train_accuracy: 0.8906, test_Accuracy: 0.9290
Epoch: [ 0] [   66/  468] time: 32.3369, train_loss: 0.25522113, train_accuracy: 0.9062, test_Accuracy: 0.9296
Epoch: [ 0] [   67/  468] time: 32.7040, train_loss: 0.09179387, train_accuracy: 0.9766, test_Accuracy: 0.9294
Epoch: [ 0] [   68/  468] time: 33.1737, train_loss: 0.19315121, train_accuracy: 0.9531, test_Accuracy: 0.9303
Epoch: [ 0] [   69/  468] time: 33.6155, train_loss: 0.14338174, train_accuracy: 0.9531, test_Accuracy: 0.9304
Epoch: [ 0] [   70/  468] time: 34.1311, train_loss: 0.20077741, train_accuracy: 0.9375, test_Accuracy: 0.9301
Epoch: [ 0] [   71/  468] time: 34.5810, train_loss: 0.11097391, train_accuracy: 0.9688, test_Accuracy: 0.9307
E

...

Epoch: [ 0] [  399/  468] time: 186.2413, train_loss: 0.15179449, train_accuracy: 0.9609, test_Accuracy: 0.9634
Epoch: [ 0] [  400/  468] time: 186.7310, train_loss: 0.15706137, train_accuracy: 0.9531, test_Accuracy: 0.9642
Epoch: [ 0] [  401/  468] time: 187.1598, train_loss: 0.08192533, train_accuracy: 0.9922, test_Accuracy: 0.9645
Epoch: [ 0] [  402/  468] time: 187.5668, train_loss: 0.04895385, train_accuracy: 0.9766, test_Accuracy: 0.9647
Epoch: [ 0] [  403/  468] time: 188.0185, train_loss: 0.07554302, train_accuracy: 0.9688, test_Accuracy: 0.9655
Epoch: [ 0] [  404/  468] time: 188.3907, train_loss: 0.04106442, train_accuracy: 0.9922, test_Accuracy: 0.9665
Epoch: [ 0] [  405/  468] time: 188.8453, train_loss: 0.12776540, train_accuracy: 0.9609, test_Accuracy: 0.9664
Epoch: [ 0] [  406/  468] time: 189.3081, train_loss: 0.07478881, train_accuracy: 0.9766, test_Accuracy: 0.9669
Epoch: [ 0] [  407/  468] time: 189.7679, train_loss: 0.13243388, train_accuracy: 0.9609, test_Accuracy:

Epoch: [ 0] [  415/  468] time: 193.0851, train_loss: 0.07469508, train_accuracy: 0.9844, test_Accuracy: 0.9642
Epoch: [ 0] [  416/  468] time: 193.4779, train_loss: 0.07268637, train_accuracy: 0.9766, test_Accuracy: 0.9640
Epoch: [ 0] [  417/  468] time: 193.9490, train_loss: 0.02599848, train_accuracy: 1.0000, test_Accuracy: 0.9643
Epoch: [ 0] [  418/  468] time: 194.3077, train_loss: 0.11360282, train_accuracy: 0.9609, test_Accuracy: 0.9643
Epoch: [ 0] [  419/  468] time: 194.8164, train_loss: 0.11903740, train_accuracy: 0.9766, test_Accuracy: 0.9649
Epoch: [ 0] [  420/  468] time: 195.2941, train_loss: 0.04823671, train_accuracy: 0.9688, test_Accuracy: 0.9663
Epoch: [ 0] [  421/  468] time: 195.7678, train_loss: 0.12258317, train_accuracy: 0.9531, test_Accuracy: 0.9671
Epoch: [ 0] [  422/  468] time: 196.2416, train_loss: 0.09682243, train_accuracy: 0.9688, test_Accuracy: 0.9670
Epoch: [ 0] [  423/  468] time: 196.7672, train_loss: 0.05357219, train_accuracy: 0.9844, test_Accuracy:

Epoch: [ 0] [  431/  468] time: 200.3726, train_loss: 0.05856483, train_accuracy: 0.9766, test_Accuracy: 0.9682
Epoch: [ 0] [  432/  468] time: 200.8223, train_loss: 0.04939337, train_accuracy: 0.9844, test_Accuracy: 0.9684
Epoch: [ 0] [  433/  468] time: 201.2342, train_loss: 0.10187678, train_accuracy: 0.9688, test_Accuracy: 0.9685
Epoch: [ 0] [  434/  468] time: 201.6821, train_loss: 0.10829489, train_accuracy: 0.9609, test_Accuracy: 0.9687
Epoch: [ 0] [  435/  468] time: 202.1428, train_loss: 0.05602003, train_accuracy: 0.9922, test_Accuracy: 0.9692
Epoch: [ 0] [  436/  468] time: 202.6275, train_loss: 0.11229108, train_accuracy: 0.9766, test_Accuracy: 0.9695
Epoch: [ 0] [  437/  468] time: 203.1072, train_loss: 0.06941544, train_accuracy: 0.9688, test_Accuracy: 0.9692
Epoch: [ 0] [  438/  468] time: 203.5939, train_loss: 0.05171809, train_accuracy: 0.9922, test_Accuracy: 0.9697
Epoch: [ 0] [  439/  468] time: 204.0646, train_loss: 0.10198402, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  447/  468] time: 207.6860, train_loss: 0.01875602, train_accuracy: 1.0000, test_Accuracy: 0.9643
Epoch: [ 0] [  448/  468] time: 208.1368, train_loss: 0.08660544, train_accuracy: 0.9766, test_Accuracy: 0.9641
Epoch: [ 0] [  449/  468] time: 208.5796, train_loss: 0.05691389, train_accuracy: 0.9844, test_Accuracy: 0.9638
Epoch: [ 0] [  450/  468] time: 209.0525, train_loss: 0.08407241, train_accuracy: 0.9844, test_Accuracy: 0.9647
Epoch: [ 0] [  451/  468] time: 209.4842, train_loss: 0.03811953, train_accuracy: 0.9766, test_Accuracy: 0.9658
Epoch: [ 0] [  452/  468] time: 209.9858, train_loss: 0.12458958, train_accuracy: 0.9609, test_Accuracy: 0.9668
Epoch: [ 0] [  453/  468] time: 210.4127, train_loss: 0.12691176, train_accuracy: 0.9609, test_Accuracy: 0.9667
Epoch: [ 0] [  454/  468] time: 210.8814, train_loss: 0.06777351, train_accuracy: 0.9766, test_Accuracy: 0.9674
Epoch: [ 0] [  455/  468] time: 211.2863, train_loss: 0.10793243, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  463/  468] time: 215.0164, train_loss: 0.06473928, train_accuracy: 0.9688, test_Accuracy: 0.9659
Epoch: [ 0] [  464/  468] time: 215.4053, train_loss: 0.08083286, train_accuracy: 0.9844, test_Accuracy: 0.9660
Epoch: [ 0] [  465/  468] time: 215.8990, train_loss: 0.05046930, train_accuracy: 0.9844, test_Accuracy: 0.9664
Epoch: [ 0] [  466/  468] time: 216.3858, train_loss: 0.08675952, train_accuracy: 0.9609, test_Accuracy: 0.9663
Epoch: [ 0] [  467/  468] time: 216.9154, train_loss: 0.06873712, train_accuracy: 0.9766, test_Accuracy: 0.9671


### Accuracy : 97.51%